In [1]:
import torch
import numpy as np
import sys
from pathlib import Path
import os
assert "HF_TOKEN" in os.environ
project_root = Path().resolve().parent
sys.path.append(str(project_root))


from src.models.hubert_wrapper import HubertProtoNet
from src.datasets.registry import DATASET_REGISTRY
from src.methods.fewshot.fewshot import create_episode, evaluate_fewshot
from src.utils.config import load_yaml

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [6]:
cfg = load_yaml(project_root / "configs/datasets/speech_commands.yaml")
dataset = DATASET_REGISTRY[cfg["name"]](cfg)

print(f"{len(dataset)} classes loaded")
for k in dataset:
    print(k, len(dataset[k]))


Loading Speech Commands: 100%|██████████| 105829/105829 [02:02<00:00, 866.60it/s] 

6 classes loaded
down 3917
go 3880
left 3801
right 3778
stop 3872
up 3723


Sanity check

In [7]:
for k in dataset:
    print(k, type(dataset[k][0]), len(dataset[k][0]))


down <class 'numpy.ndarray'> 16000
go <class 'numpy.ndarray'> 11889
left <class 'numpy.ndarray'> 16000
right <class 'numpy.ndarray'> 14336
stop <class 'numpy.ndarray'> 13375
up <class 'numpy.ndarray'> 14861


In [8]:
model = HubertProtoNet(
    hubert_model="facebook/hubert-base-ls960",
    freeze_hubert=True,
).to(device)

model.eval()


HubertProtoNet(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): HubertEncode

In [9]:
mean, std = evaluate_fewshot(
    dataset,
    model,
    device=device,
    n_way=5,
    n_shot=1,
    n_query=5,
    n_episodes=50,
)

print(f"1-shot accuracy: {mean*100:.2f}% ± {std*100:.2f}%")


5-way 1-shot:   0%|          | 0/50 [00:00<?, ?it/s]

Creating episode...
Extracting features...
Running model...


5-way 1-shot:   2%|▏         | 1/50 [00:03<02:35,  3.17s/it]

Computing prototypes and accuracy...
Creating episode...
Extracting features...
Running model...


5-way 1-shot:   4%|▍         | 2/50 [00:06<02:33,  3.20s/it]

...

Computing prototypes and accuracy...
Creating episode...
Extracting features...
Running model...


5-way 1-shot:  98%|█████████▊| 49/50 [03:13<00:03,  3.41s/it]

Computing prototypes and accuracy...
Creating episode...
Extracting features...
Running model...


5-way 1-shot: 100%|██████████| 50/50 [03:17<00:00,  3.95s/it]

Computing prototypes and accuracy...
1-shot accuracy: 47.52% ± 12.46%
